<a href="https://colab.research.google.com/github/ds4geo/ds4geo/blob/master/DS4GEO_L3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime

In [ ]:
air_temp = pd.read_excel("https://github.com/ds4geo/ds4geo/raw/master/data/timeseries/Au%C3%9Fenluft%2BEingangslabyrinth.xlsx", sheet_name="Daten3", usecols=[0,6])

In [ ]:
air_temp

In [ ]:
cave_temp = pd.read_excel("https://github.com/ds4geo/ds4geo/raw/master/data/timeseries/Au%C3%9Fenluft%2BEingangslabyrinth.xlsx", sheet_name="Daten3", usecols=[8,9])

In [ ]:
cave_temp

In [ ]:
cave_temp.plot("Date", "Eingangslaby Temp")
air_temp.plot(" ", "Temp korr")

In [ ]:
cave_co2 = pd.read_excel(r"https://github.com/ds4geo/ds4geo/raw/master/data/timeseries/CO2%20_compiled.xlsx",
                         sheet_name="Data Stream (2)", names=["time", "unknown", "d13C", "d18O", "ppm CO2", "ctime"])

In [ ]:
cave_co2

In [ ]:
fig, ax = plt.subplots(figsize=(18,10))
ax2 = ax.twinx()
air_temp.plot(" ", "Temp korr", ax=ax)
cave_temp.plot("Date", "Eingangslaby Temp", ax=ax)
cave_co2.plot("time", "ppm CO2", ax=ax2, color="r")
ax.set_xlim([datetime.date(2015,9,1), datetime.date(2016,1,1)])